<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_tensorflow2/blob/main/7_PRETRAIN_METHOD/KorQuAD_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 허깅페이스 transformers 라이브러리 가져오기

In [ ]:
pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 27.7 MB/s 
     |████████████████████████████████| 880 kB 36.5 MB/s 
     |████████████████████████████████| 3.0 MB 18.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2dadb6269ac337c47a7312ed145f9bd7fc85a56d36a253d03c27cd4685b2b44b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=a877853ed864052114b03080ebd8cecdddf9f55527874f9c46639afc2c3cbcc8
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib
import wget

MAX_LEN = 384
EPOCHS = 3
VERBOSE = 2
BATCH_SIZE = 16

In [ ]:
cd drive/MyDrive/BERT

/content/drive/MyDrive/BERT


In [ ]:
DATA_OUT_PATH = './data_out'

In [ ]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [ ]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

3891200/3881058 [==============================] - 0s 0us/step


In [ ]:
'''
with open(train_path) as f:
  raw_train_data=json.load(f)

raw_train_data
'''

'\nwith open(train_path) as f:\n  raw_train_data=json.load(f)\n\nraw_train_data\n'

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

'./bert-base-multilingual-cased//bert-base-multilingual-cased-config.json'

In [ ]:
!mv ./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json ./bert-base-multilingual-cased/config.json

In [ ]:
!wget -P ./bert-base-multilingual-cased/ https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5

--2022-09-07 09:51:41--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.164.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.164.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1083389348 (1.0G) [binary/octet-stream]
Saving to: ‘./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5’

bert-base-multiling 100%[===================>]   1.01G  22.7MB/s    in 31s     

2022-09-07 09:52:13 (33.3 MB/s) - ‘./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5’ saved [1083389348/1083389348]



In [ ]:
!mv ./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5 ./bert-base-multilingual-cased/tf_model.h5

In [ ]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters  
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        # 텍스트를 .encode(text)하면 나오는 sequence는 맨 앞에 [CLS]와 맨 뒤에 [SEP] 토큰이 붙어서 나온다. 
        # 그러므로, 두개의 encode된 ids를 이어 붙이기 위해서는 뒷 sequence의 [CLS]를 없에주어야 한다. 그래서 [1:]을 통해 0번째가 아닌 1번째 index부터 맨 뒤까지 슬라이싱한다.
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]   
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(    
            tokenized_question.ids[1:] 
        )                                                                  
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],  # squad_example 개수 x MAX_LEN
        dataset_dict["token_type_ids"], # squad_example 개수 x MAX_LEN
        dataset_dict["attention_mask"], # squad_example 개수 x MAX_LEN
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]] # 각각 squad_example 개수 x 1
    return x, y

In [ ]:
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

In [ ]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [ ]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [ ]:
def normalized_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [ ]:
class ExactMatch(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):   # 기본적으로 offsets의 길이는 context를 BertWordPieceTokenizer를 이용해 토큰 분리한 개수 + 2 
                continue               #( 맨 앞과 끝에 (0,0)이 존재 : [CLS]와[SEP]용도) => start==len(offsets)이면 (0,0)이 정답의 start라는 것이여서 안됨.=> 범위에 포함
            pred_char_start = offsets[start][0] #기존 context에서 token의 시작 위치 ind          
            if end < len(offsets):  # 마지막 (0,0)을 포함하지 않는다.
                pred_char_end = offsets[end][1] # 기존 context에서 정답 마지막 token의 위치 + 1  : +1인 이유는 파이썬 열린구간 범위계산은 
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]     # (a,b) => a<= x < b 이므로 맨 뒤 하나를 포함시키지 않아서 offsets에서도 b값은 +1 되어있다.
            else:
                pred_ans = squad_eg.context[pred_char_start:]    # end < len(offsets) 이 아닌경우는 맨 마지막 (0,0)을  포함하는 경우이기 때문에 
                                                                  # 시작지점부터 context 끝까지 모두 정답으로 판단한다.
            normalized_pred_ans = normalized_answer(pred_ans)
            normalized_true_ans = normalized_answer(squad_eg.answer_text)
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])   # 지표 : 정확도(accuracy)   정답/실제
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [ ]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [ ]:
model_name = "tf2_bert_korquad"

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:

history = korquad_model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,  # For demonstration, 3 epochs are recommended
    verbose=VERBOSE,
    batch_size=BATCH_SIZE,
    callbacks=[exact_match_callback, cp_callback]
)

In [ ]:
print(history.history)

In [ ]:
def plot_graphs(history, string, string_1, string_2):
    # loss 
    plt.plot(history.history[string])
    plt.plot(history.history[string_1])
    plt.plot(history.history[string_2])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string_1, string_2])
    plt.show()

In [ ]:
plot_graphs(history, 'loss', 'output_1_loss', 'output_2_loss')

### Test

In [ ]:
a='나는 nlp를 공부하고 있는 사람이다.'

In [ ]:
tokenizer.encode(a)

Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode(a).tokens

['[CLS]',
 '나는',
 'nl',
 '##p',
 '##를',
 '공',
 '##부',
 '##하고',
 '있는',
 '사람이',
 '##다',
 '.',
 '[SEP]']

In [ ]:
tokenizer.encode(a).offsets

[(0, 0),
 (0, 2),
 (3, 5),
 (5, 6),
 (6, 7),
 (8, 9),
 (9, 10),
 (10, 12),
 (13, 15),
 (16, 19),
 (19, 20),
 (20, 21),
 (0, 0)]

In [ ]:
tokenizer.encode(a).ids

[101,
 100585,
 21517,
 10410,
 11513,
 8896,
 14646,
 12453,
 13767,
 97802,
 11903,
 119,
 102]